In [ ]:
# # Add the root directory to the path to allow importing the module
# import sys
# sys.path.append('/mn/stornext/u3/avijeetp/codes/ISPy')
# sys.path.append('/mn/stornext/u3/avijeetp/codes/pyMilne')
# sys.path.append('/mn/stornext/u3/avijeetp/codes/helita')
# sys.path.append('/mn/stornext/d9/data/avijeetp/1_Projects/33_SST/LaPalma')

In [ ]:
import numpy as np
import MilneEddington as ME
import crisp
import time
import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero")
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero")
import inv_utils as iu
from scipy.ndimage import median_filter
from helita.io import lp
from lp_scripts.get_fov_angle import fov_angle
from hmi_plot import plot_hmi_ic_mag

In [ ]:
import importlib
importlib.reload(iu)
print('inv_utils reloaded') 

In [ ]:
### Small scale anemone jet 

datadir = '/mn/stornext/d18/lapalma/reduc/2024/2024-05-21/CRISP/cubes_nb/'
crisp_im = datadir + 'nb_6173_2024-05-21T10:19:04_10:19:04=0-52_stokes_corrected_im.fits'
sst_blos_file = datadir + 'Blos.6173_2024-05-21T10:19:04.icube'
sst_bhor_file = datadir + 'Bhor.6173_2024-05-21T10:19:04.icube'
xorg = 1110
xsize = 800
yorg = 570
ysize = 800
tt = 8
is_north_up = True
scale = 0.044 # arcsec/pixel
crop = True

In [ ]:
### Sunspot data set close to center also observed with Hinode

# datadir = '/mn/stornext/d18/lapalma/reduc/2020/2020-08-07/CRISP/cubes_nb/'
# crisp_im = datadir + 'nb_6173_2020-08-07T08:22:14_scans=0-56_stokes_corrected_im.fits'
# xorg = 200
# xsize = 480
# yorg = 360
# ysize = 400
# tt = 0
# scale = 0.058 # arcsec/pixel
# is_north_up = False
# crop = False

In [ ]:
### QS dataset used by Aditi

# datadir = '/mn/stornext/d18/lapalma/reduc/2021/2021-06-22/CRISP/cubes_nb/'
# crisp_im = datadir + 'nb_6173_2021-06-22T08:17:48_scans=0-162_stokes_corrected_im.fits'
# blos_cube = datadir + 'Blos.6173_2021-06-22T08:17:48.icube'
# bhor_cube = datadir + 'Bhor.6173_2021-06-22T08:17:48.icube'
# xsize = 256
# ysize = 256
# xorg = 273
# yorg = 420
# tt = 41
# scale = 0.058 # arcsec/pixel
# crop = False
# is_north_up = False

In [ ]:

xrange = [xorg, xorg + xsize]
yrange = [yorg, yorg + ysize]

In [ ]:
fits_info = iu.get_fits_info(crisp_im)
t_obs = fits_info['avg_time_obs']
fov = fov_angle(t_obs)
print(f'FOV angle: {fov:.2f} deg')

In [ ]:
ic_series = 'hmi.Ic_45s'
mag_series = 'hmi.M_45s'
email = 'avijeet.prasad@astro.uio.no'
save_dir = 'temp/'

In [ ]:
nx = fits_info['nx']
ny = fits_info['ny']
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]


In [ ]:
fits_info.keys()

In [ ]:
# x1 = fits_info['ln_min']
# x2 = fits_info['ln_max']
# y1 = fits_info['lt_min']
# y2 = fits_info['lt_max']

In [ ]:
x_list = np.linspace(x1, x2, num=nx)
y_list = np.linspace(y1, y2, num=ny)
if crop:
    x_list = x_list[xrange[0]:xrange[1]]
    y_list = y_list[yrange[0]:yrange[1]]
    x1 = x_list[0]
    x2 = x_list[-1]
    y1 = y_list[0]
    y2 = y_list[-1]
    nx = xsize
    ny = ysize
print(f'x1: {x1:.2f}, x2: {x2:.2f}, y1: {y1:.2f}, y2: {y2:.2f}')
print(f'nx: {nx}, ny: {ny}')

In [ ]:
tobs = fits_info['all_start_times'][tt]

In [ ]:
plot_hmi_ic_mag(tobs, ic_series, mag_series, email, x1, x2, y1, y2, draw_rectangle=False, height=56, width=56, rot_fov=fov, save_dir=save_dir, draw_circle=False, radius=87, enhance_ic=False, figsize=(10, 5), overwrite=False, buffer=5)

In [ ]:
iu.plot_image(crisp_im, ss=0, figsize=(6,6), fontsize=8, rot_fov=fov, north_up=not(is_north_up), crop=crop, xrange=xrange, yrange=yrange, xtick_range=[x1,x2], ytick_range=[y1,y2])

In [ ]:

tt = 0
mu = fits_info['mu']
ww = fits_info['nw'] // 2
ss = 0

iu.plot_image(crisp_im, xrange=xrange, yrange=yrange, ww=ww, tt=tt, ss=ss, crop=crop, figsize=(5,5), fontsize=8, rot_fov=fov, north_up=False,xtick_range=[x1,x2],ytick_range=[y1,y2])

In [ ]:
# blos_sst = lp.getdata(blos_cube)
# bhor_sst = lp.getdata(bhor_cube)

In [ ]:
#
# Decide to work in float32 or float64
#
dtype = 'float32'
nthreads = 96
#
# Load data, wavelength array and cmap
#
class container:
    def __init__(self):
        pass
l = container()
container.iwav = iu.getWavelengths(crisp_im)
container.d = iu.loadFits(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 
container.cmap = iu.loadCmap(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 
mask = iu.get_nan_mask(crisp_im, tt=tt,crop=crop, xrange=xrange, yrange=yrange) 
print(mask.shape)
iu.plot_image(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 

In [ ]:

# Minimum step:
dw = np.min(np.diff(l.iwav))
# dw = round((lambda*10. - lc) * 1000.) ; offset in mA
dw = round(dw*1000.)/1000. # avoid floating point errors

# ==============================================================================

# The inversions need to account for the instrumental
# profile, which involve convolutions. The convolutions
# must be done in a wavelength grid that is at least
# 1/2 of the FWHM of the instrumental profile. In the
# case of CRISP that would be ~55 mA / 2 = ~27.5 mA
#
# Get finer grid for convolutions purposes
# Since we only observed at the lines, let's create
# two regions, one for each line
#
# The observed line positions are not equidistant, the
# Fe I 6301 points only fit into a regular grid of 5 mA
# whereas the Fe I 6302 can fit into a 15 mA grid
#
iw, idx = iu.findgrid(l.iwav, dw)  # Fe I 6173

# ==============================================================================


#
# Now we need to create a data cube with the fine grid
# dimensions. All observed points will contribute to the
# inversion. The non-observed ones will have zero weight
# but will be used internally to properly perform the
# convolution of the synthetic spectra
#


ny, nx = l.d.shape[0:2]
obs = np.zeros((ny, nx, 4, iw.size), dtype=dtype, order='c')

for ss in range(4):
    for ii in range(idx.size):
        obs[:, :, ss, idx[ii]] = l.d[:, :, ss, ii]

# ==============================================================================
#
# Create sigma array with the estimate of the noise for
# each Stokes parameter at all wavelengths. The extra
# non-observed points will have a very large noise (1.e34)
# (zero weight) compared to the observed ones (3.e-3)
# Since the amplitudes of Stokes Q,U and V are very small
# they have a low imprint in Chi2. We can artificially
# give them more weight by lowering the noise estimate.
#
sig = np.zeros((4, iw.size), dtype=dtype) + 1.e32
sig[:, idx] = 5.e-3
sig[1:3, idx] /= 9.0
sig[3, idx] /= 4.0

# ==============================================================================
#
# Init Me class. We need to create two regions with the
# wavelength arrays defined above and a instrumental profile
# for each region in with the same wavelength step
#
tw = (np.arange(iw.size, dtype=dtype)-iw.size//2)*dw

# ==============================================================================
# Central wavelength of the line:
l0 = iw[iw.size//2]
tr = crisp.crisp(l0).dual_fpi(tw, erh=-0.001)

regions = [[iw, tr/tr.sum()]]
lines = [int(l0)]
me = ME.MilneEddington(regions, lines, nthreads=nthreads, precision=dtype)

In [ ]:

#
# Init model parameters
#
iPar = np.float64([1500, 2.2, 1.0, -0.5, 0.035, 50., 0.1, 0.24, 0.7]) # [B_tot, theta_B, chi_B, gamma_B, v_los, eta_0, Doppler width, damping, s0, s1]
Imodel = me.repeat_model(iPar, ny, nx)


In [ ]:
#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations) of simple pixel-wise inversion
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(Imodel, obs, sig, nRandom=6, nIter=50, chi2_thres=1, mu=mu)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))
iu.plot_output(Imodel,mask,scale=scale, save_fig=False)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
# Median filter for all except for the azimuth angle:
size_filter = 21
for ii in range(Imodel.shape[2]):
    if ii == 2:
        sin2azi = np.sin(Imodel[:, :, 2]*2.0)
        cos2azi = np.cos(Imodel[:, :, 2]*2.0)
        Imodel[:, :, 2] = 0.5*np.arctan2(median_filter(sin2azi, size=(size_filter,size_filter)),median_filter(cos2azi, size=(size_filter,size_filter)))
        Imodel[:,:,2][Imodel[:,:,2] < 0] += np.pi
    else:
        Imodel[:, :, ii] = median_filter(Imodel[:, :, ii], size=(size_filter,size_filter))

iu.plot_output(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
#
# Run second cycle
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(Imodel, obs, sig, nRandom=2, nIter=50, chi2_thres=1, mu=mu)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))
iu.plot_output(Imodel,mask,scale=scale, save_fig=False)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
import matplotlib.pyplot as plt
import copy
mos2 = copy.deepcopy(Imodel)
# Create a new figure for Blos and Bhor maps
fig2, ax2 = plt.subplots(nrows=1, ncols=1, figsize=(8,8))
# Blos map
Blos = mos2[:, :, 0] * np.cos(mos2[:, :, 1])
Blos[mask] = 1.01 * np.percentile(Blos[~mask], 99)
Blos = iu.make_north_up(Blos, 0)
vmin = np.percentile(Blos, 1)
vmax = np.percentile(Blos, 99)
im1 = ax2.imshow((Blos), cmap='Greys_r', interpolation='nearest',
                    aspect='equal', vmin=vmin, vmax=vmax, origin='lower')
ax2.tick_params(axis='both', which='major', labelsize=14)
cbar1 = fig2.colorbar(im1, ax=ax2, orientation='horizontal', shrink=0.8, pad=0.05)
cbar1.set_label('Blos [G]', fontsize=18)
cbar1.ax.tick_params(labelsize=14)
plt.show()

In [ ]:

#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations)
#
t0 = time.time()
alphas=np.float32([2, 0.5, 2, 0.01, 0.1, 0.01, 0.1, 0.01, 0.01])
mo, syn, chi2 = me.invert_spatially_regularized(Imodel, obs, sig,  nIter=50, chi2_thres=1, mu=mu, alpha=30., alphas=alphas, method=1, delay_bracket=3)
t1 = time.time()
print("dT = {0}s -> <Chi2> (including regularization) = {1}".format(t1-t0, chi2))

#
# Correct velocities for cavity error map from CRISP
#
mos = np.squeeze(mo) # Remove the singleton dimension in the model and make the shape (ny, nx, 9) from (1, ny, nx, 9)
mos[:,:,3] += (l.cmap * 10) / l0 * 2.9e5
# mos[:,:,3] += l.cmap+0.45 # The 0.45 is a global offset that seems to make the umbra at rest


In [ ]:
iu.plot_output(mos,mask,scale=scale)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False,vmax=5)

In [ ]:
datadir

In [ ]:
blos_sst = lp.getdata(sst_blos_file)
bhor_sst = lp.getdata(sst_bhor_file)

In [ ]:
fits_info['nx']

In [ ]:
blos_sst.shape

In [ ]:
blos_sst_crop = blos_sst[xrange[0]:xrange[1], yrange[0]:yrange[1], 0].T
bhor_sst_crop = bhor_sst[xrange[0]:xrange[1], yrange[0]:yrange[1], 0].T

In [ ]:
import matplotlib.pyplot as plt
# Create a new figure for Blos and Bhor maps
fig2, ax2 = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
vmin = np.percentile(blos_sst_crop, 1)
vmax = np.percentile(blos_sst_crop, 99)
im1 = ax2[0].imshow(blos_sst_crop, cmap='Greys_r', interpolation='nearest',
                    aspect='equal', vmin=vmin, vmax=vmax, origin='lower')
ax2[0].tick_params(axis='both', which='major', labelsize=14)
cbar1 = fig2.colorbar(im1, ax=ax2[0], orientation='horizontal', shrink=0.8, pad=0.05)
cbar1.set_label('Blos [G]', fontsize=18)
cbar1.ax.tick_params(labelsize=14)

# Bhor map
im2 = ax2[1].imshow(bhor_sst_crop, cmap='Greys_r', interpolation='nearest',
                    aspect='equal', origin='lower')
ax2[1].tick_params(axis='both', which='major', labelsize=14)
cbar2 = fig2.colorbar(im2, ax=ax2[1], orientation='horizontal', shrink=0.8, pad=0.05)
cbar2.set_label('Bhor [G]', fontsize=18)
cbar2.ax.tick_params(labelsize=14)

fig2.tight_layout()

plt.show()
